# FM Signal Strength Prediction: Why Distance Alone Fails

**SBE NYC Chapter 15 - RadioLand Demo Notebook**

---

In this notebook, we'll explore why predicting FM radio signal strength is more complex than just measuring distance from the transmitter. We'll:

1. Calculate distance-based predictions (the naive approach)
2. See how terrain completely changes the picture
3. Understand why RadioLand uses the Longley-Rice propagation model

**Run each cell in order by pressing Shift+Enter**

## Step 1: Setup

First, let's import the libraries we need and define some helper functions.

In [ ]:
import math
import pandas as pd
import matplotlib.pyplot as plt

# For nice plotting
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 6]

print("Libraries loaded!")

## Step 2: The Haversine Formula

To calculate the distance between two points on Earth (given latitude/longitude), we use the **Haversine formula**. This accounts for Earth's curvature.

In [ ]:
def haversine_km(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points on Earth.
    Returns distance in kilometers.
    """
    R = 6371  # Earth's radius in km
    
    # Convert to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    return R * c

# Test it: NYC to Philadelphia
nyc_to_philly = haversine_km(40.7128, -74.0060, 39.9526, -75.1652)
print(f"NYC to Philadelphia: {nyc_to_philly:.1f} km ({nyc_to_philly * 0.621:.1f} miles)")

## Step 3: Sample Station Data

Let's define some real FM stations in the NYC area. This is actual FCC data.

In [ ]:
# Real NYC-area FM stations - data from FCC database (November 2025)
# Note: These coordinates and technical parameters are from actual FCC filings

stations = [
    # Callsign, Frequency, City, TX Lat, TX Lon, ERP (kW), HAAT (m), Class
    ("WCBS-FM", 101.1, "New York (Empire State Bldg)", 40.7484, -73.9857, 6.7, 408, "B"),
    ("WNYC-FM", 93.9, "New York (Empire State Bldg)", 40.7484, -73.9857, 5.2, 415, "B"),
    ("WPLJ", 95.5, "New York (Empire State Bldg)", 40.7484, -73.9857, 6.7, 408, "B"),
    ("WFUV", 90.7, "New York (Fordham)", 40.8801, -73.8774, 47.0, 155, "B"),
    ("WFMU", 91.1, "East Orange, NJ", 40.7887, -74.2551, 1.25, 151, "A"),
    ("WNNJ", 103.7, "Newton, NJ", 41.1868, -74.7674, 2.3, 272, "B1"),
    ("W289BE", 105.7, "Ellenville, NY", 41.6064, -74.5294, 0.2, 150, "D"),  # Translator
]

df_stations = pd.DataFrame(stations, 
    columns=['Callsign', 'Frequency', 'City', 'TX_Lat', 'TX_Lon', 'ERP_kW', 'HAAT_m', 'Class'])

print("Station Data Loaded (from FCC database):\n")
df_stations

## Step 4: The Naive Approach - Distance Only

A simple approach would be: **closer stations = stronger signals**. Let's test this from a listener location in the mountains of northwest New Jersey.

In [ ]:
# Listener location: Sparta, NJ (Sussex County - C.R. 517 near Sparta Middle School)
listener_lat = 41.0569
listener_lon = -74.6094
listener_name = "Sparta, NJ (Sussex County)"

print(f"Listener Location: {listener_name}")
print(f"   Coordinates: {listener_lat}, {listener_lon}\n")

# Calculate distance to each station
df_stations['Distance_km'] = df_stations.apply(
    lambda row: haversine_km(listener_lat, listener_lon, row['TX_Lat'], row['TX_Lon']), 
    axis=1
)

df_stations['Distance_mi'] = df_stations['Distance_km'] * 0.621371

# Sort by distance
df_by_distance = df_stations.sort_values('Distance_km')

print("Stations sorted by distance (naive prediction - closer = better):")
df_by_distance[['Callsign', 'Frequency', 'City', 'Distance_mi', 'ERP_kW']].round(1)

## Step 5: A Slightly Better Model - Consider Power

FM stations don't all broadcast at the same power. A Class B station with 6.7 kW covers much more area than a Class D translator with 200 watts.

Let's use a simplified **free-space path loss** model that considers both distance AND power:

$$\text{Signal (dBu)} \approx 10 \cdot \log_{10}(\text{ERP in watts}) - 20 \cdot \log_{10}(\text{distance in km}) + C$$

In [ ]:
def simple_signal_prediction(erp_kw, distance_km, freq_mhz=100):
    """
    Simplified signal strength prediction based on free-space path loss.
    Returns estimated field strength in dBu (dB above 1 microvolt/meter).
    
    This is a ROUGH approximation - real propagation is much more complex!
    """
    if distance_km < 1:
        distance_km = 1  # Avoid log(0)
    
    erp_watts = erp_kw * 1000
    
    # Simplified formula (approximation of FCC curves for flat terrain)
    # Reference: 1 kW at 1 km ~ 100 dBu for FM frequencies
    signal_dbu = 106.9 + 10 * math.log10(erp_watts / 1000) - 20 * math.log10(distance_km)
    
    return signal_dbu

# Calculate predicted signal for each station
df_stations['Predicted_dBu_Simple'] = df_stations.apply(
    lambda row: simple_signal_prediction(row['ERP_kW'], row['Distance_km'], row['Frequency']),
    axis=1
)

# Sort by predicted signal strength
df_by_signal = df_stations.sort_values('Predicted_dBu_Simple', ascending=False)

print("Stations sorted by PREDICTED signal (distance + power model):\n")
print(df_by_signal[['Callsign', 'Frequency', 'Distance_mi', 'ERP_kW', 'Predicted_dBu_Simple']].round(1).to_string(index=False))

## Step 6: Reality Check - Actual Longley-Rice Predictions

Now let's see what the **Longley-Rice model** actually predicts for these same stations.

The Longley-Rice Irregular Terrain Model (ITM) considers:
- **Terrain elevation** between TX and RX (using SRTM satellite data)
- **Earth curvature** 
- **Atmospheric refraction**
- **Ground conductivity**
- **Antenna heights (HAAT)**

In [ ]:
# ACTUAL Longley-Rice predictions from RadioLand API for Sparta, NJ
# These were computed using real SRTM terrain data

actual_longley_rice = {
    "WCBS-FM": 25.4,   # Empire State Building - BLOCKED by terrain!
    "WNYC-FM": 24.6,   # Empire State Building - BLOCKED by terrain!
    "WPLJ": 25.5,      # Empire State Building - BLOCKED by terrain!
    "WFUV": 15.0,      # Fordham - even worse path
    "WFMU": 19.9,      # East Orange - blocked by ridges
    "WNNJ": 71.7,      # Newton, NJ - LOCAL, clear path!
    "W289BE": 31.9,    # Ellenville translator - terrain HELPS (downhill shot)
}

df_stations['Actual_LR_dBu'] = df_stations['Callsign'].map(actual_longley_rice)

print("Comparison: Simple Model vs Longley-Rice Reality\n")
comparison = df_stations[['Callsign', 'Frequency', 'Distance_mi', 'ERP_kW', 'Predicted_dBu_Simple', 'Actual_LR_dBu']].copy()
comparison['Error_dB'] = comparison['Predicted_dBu_Simple'] - comparison['Actual_LR_dBu']
comparison = comparison.sort_values('Actual_LR_dBu', ascending=False)

print(comparison.round(1).to_string(index=False))

## Step 7: Visualize the Difference

Let's plot this to really see how wrong the simple model can be!

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sort by actual signal for consistent display
plot_data = df_stations.sort_values('Actual_LR_dBu', ascending=True)

# Left plot: Bar comparison
x = range(len(plot_data))
width = 0.35

bars1 = axes[0].barh([i - width/2 for i in x], plot_data['Predicted_dBu_Simple'], 
                      width, label='Simple (Distance+Power)', color='#ff6b6b', alpha=0.8)
bars2 = axes[0].barh([i + width/2 for i in x], plot_data['Actual_LR_dBu'], 
                      width, label='Longley-Rice (Terrain)', color='#4ecdc4', alpha=0.8)

axes[0].set_yticks(x)
axes[0].set_yticklabels([f"{row['Callsign']} ({row['Frequency']})" for _, row in plot_data.iterrows()])
axes[0].set_xlabel('Signal Strength (dBu)')
axes[0].set_title(f'Signal Predictions at {listener_name}')
axes[0].legend(loc='lower right')
axes[0].axvline(x=60, color='green', linestyle='--', alpha=0.5, label='Good signal (60 dBu)')
axes[0].axvline(x=40, color='orange', linestyle='--', alpha=0.5, label='Weak signal (40 dBu)')

# Right plot: Error by distance
colors = ['#e74c3c' if err > 0 else '#27ae60' for err in (df_stations['Predicted_dBu_Simple'] - df_stations['Actual_LR_dBu'])]
axes[1].scatter(df_stations['Distance_mi'], 
                df_stations['Predicted_dBu_Simple'] - df_stations['Actual_LR_dBu'],
                s=150, c=colors, alpha=0.7, edgecolors='black')

for _, row in df_stations.iterrows():
    axes[1].annotate(row['Callsign'], 
                     (row['Distance_mi'], row['Predicted_dBu_Simple'] - row['Actual_LR_dBu']),
                     xytext=(5, 5), textcoords='offset points', fontsize=9)

axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[1].set_xlabel('Distance (miles)')
axes[1].set_ylabel('Prediction Error (Simple - Actual) dB')
axes[1].set_title('How Wrong is the Simple Model?')
axes[1].text(0.02, 0.98, 'Red = Overestimate\nGreen = Underestimate', 
             transform=axes[1].transAxes, fontsize=9, verticalalignment='top')

plt.tight_layout()
plt.show()

print("\nNotice: The simple model OVERESTIMATES signals from NYC (Empire State Building)")
print("   by 30-40 dB because it can't account for mountain terrain blocking the path!")
print("\n   Meanwhile, the 200-watt translator W289BE does BETTER than predicted")
print("   because it has a favorable downhill terrain path to Sparta.")



### What's in the way?

The **Appalachian foothills** of northwestern New Jersey:
- Green Pond Mountain
- Kanouse Mountain  
- Hamburg Mountain

These ridges rise to 1,200+ feet and **block the signal path** from Manhattan.

### The Flip Side: Terrain Can Help!

**W289BE** is just a 200-watt translator in Ellenville, NY - yet it outperforms the prediction because it has a clear downhill shot into the Sparta area. The terrain works in its favor!

In [ ]:
# Let's show what RadioLand considers "listenable"
print("FM Signal Quality Thresholds:\n")
print("  >= 60 dBu  ->  City-grade (reliable indoor reception)")
print("  >= 50 dBu  ->  Suburban (good with decent antenna)")
print("  >= 40 dBu  ->  Fringe (outdoor antenna needed)")
print("  <  40 dBu  ->  Unlikely to receive reliably")
print("\n" + "="*55)

print(f"\nAt {listener_name}, Longley-Rice predicts:\n")

for _, row in df_stations.sort_values('Actual_LR_dBu', ascending=False).iterrows():
    signal = row['Actual_LR_dBu']
    if signal >= 60:
        quality = "CITY-GRADE"
    elif signal >= 50:
        quality = "Suburban"
    elif signal >= 40:
        quality = "Fringe"
    else:
        quality = "Unlikely"
    
    print(f"  {row['Callsign']:8} {row['Frequency']:5.1f} MHz -> {signal:5.1f} dBu  [{quality:11}]  ({row['Distance_mi']:.0f} mi, {row['ERP_kW']} kW)")

## Conclusion

1. **Terrain matters enormously** - Mountains, hills, and ridges can block signals completely, turning a 6.7 kW Class B into background noise

2. **Distance and power aren't enough** - A simple model can be off by **30-40 dB** (that's 1,000x to 10,000x in power terms!)

3. **Terrain can help too** - A favorable path can make a 200W translator stronger than expected

4. **Real-world propagation is complex** - The Longley-Rice model (developed in the 1960s) accounts for irregular terrain using elevation data

5. **Lonhley-Rice SRTM data** - Shuttle Radar Topography Mission provides ~30m resolution terrain data for the entire US

---

### Learn More

- **SPLAT! (Open source Longley-Rice)**: splat.ecok.edu
- **FCC Station Data**: fcc.gov/media/radio/lms

---


## Bonus: Try Your Own Location!

Change the coordinates below and re-run to see distance-based predictions for a different spot.

(Note: For actual Longley-Rice predictions, you'd need to use RadioLand or run SPLAT!)

In [ ]:
# ========================================
# TRY YOUR OWN LOCATION!
# ========================================
# Change these coordinates and re-run this cell

my_lat = 40.7580   # Example: Times Square
my_lon = -73.9855
my_name = "Times Square, NYC"

# ========================================

print(f"\nDistance from {my_name} to each station:\n")

for _, row in df_stations.iterrows():
    dist = haversine_km(my_lat, my_lon, row['TX_Lat'], row['TX_Lon'])
    dist_mi = dist * 0.621371
    predicted = simple_signal_prediction(row['ERP_kW'], dist)
    print(f"  {row['Callsign']:8} {row['Frequency']:5.1f} MHz -> {dist_mi:5.1f} mi -> ~{predicted:.0f} dBu (simple model)")

